# Creacion catalogo

In [1]:
import pandas as pd

In [9]:
catalogo = pd.read_excel('../../data/catalogo_productos.xlsx').rename(columns={'Sku':'sku', 'Familia':'familia', 'Marca':'marca', 'Descripción':'descripcion'})
datos = pd.read_parquet('../../data/data_for_qlik.parquet')

In [40]:
def limpia_cadena(cadena:str):
    if cadena is None:
        return 'SIN DESCRIPCION'
    cadena = cadena.replace(',', ' ')
    cadena = cadena.strip()
    return cadena

In [49]:
def selecciona_descripcion(lista_descripciones:list):
    '''
    Esta función recibe una lista con todas las descripciones
    que tiene un producto dado y elige la mejor (la más corta)
    Si en alguna hay una descripción del tipo "SIN DESCRIPCION"
    se omite en caso de que haya otras
    '''
    if len(lista_descripciones) == 1:
        return lista_descripciones[0]
    if "SIN DESCRIPCION" in lista_descripciones:
        lista_descripciones.remove("SIN DESCRIPCION")
    return min(lista_descripciones, key=len)

In [78]:
catalogo_actualizado = (datos
 [['sku', 'descripcion']]
 .merge(catalogo, on='sku', how='outer')
 .fillna(value={'familia':'EXTERNO', 'marca':'BAYER'})
 .assign(descripcion=lambda df: df.descripcion_y.fillna(df.descripcion_x).apply(limpia_cadena))
 .drop(['descripcion_x', 'descripcion_y'], axis=1)
 # Vamos a agrupar para cada sku, contar cuántas descripciones distintas hay
 .drop_duplicates()
 .groupby(['sku', 'familia', 'marca'])
 .agg(descripciones=pd.NamedAgg('descripcion', set))
 .assign(descripcion=lambda x: x.descripciones.apply(list).apply(selecciona_descripcion))
 .drop(['descripciones'], axis=1)
 .reset_index()
)

In [80]:
catalogo_actualizado.to_parquet('../../data/static/catalogo_productos.parquet', index=False)

In [5]:
catalogo = pd.read_parquet('../../data/static/catalogo_productos.parquet')
datos = pd.read_parquet('../../data/data_for_qlik.parquet')

In [9]:
datos.query('sku == "H048"')

,sku,planeado,producido,tipo,planta,inicio_semana,inicio_semana_real,semana,anio,porcentaje,completado,superado,inferior,terminado,estatus,familia,marca,descripcion
272,H048,3500.0,3500.0,LIQUIDOS,IZUCAR,14/04/2023,16/04/2023,15,2023,1.00,1,0,0,1,COMPLETADO,HERBICIDA,DRAGOCSON,DRAGOCSON 4X5 LT
346,H048,1800.0,2900.0,LIQUIDOS,IZUCAR,05/05/2023,07/05/2023,18,2023,1.61,0,1,0,1,SUPERADO,HERBICIDA,DRAGOCSON,DRAGOCSON 4X5 LT
454,H048,1150.0,1804.0,LIQUIDOS,IZUCAR,28/04/2023,30/04/2023,17,2023,1.57,0,1,0,1,SUPERADO,HERBICIDA,DRAGOCSON,DRAGOCSON 4X5 LT
531,H048,1200.0,598.0,LIQUIDOS,IZUCAR,26/05/2023,28/05/2023,21,2023,0.50,0,0,1,0,PENDIENTE,HERBICIDA,DRAGOCSON,DRAGOCSON 4X5 LT
652,H048,6200.0,0.0,LIQUIDOS,IZUCAR,02/06/2023,04/06/2023,22,2023,0.00,0,0,1,0,PENDIENTE,HERBICIDA,DRAGOCSON,DRAGOCSON 4X5 LT
737,H048,4550.0,2207.0,LIQUIDOS,IZUCAR,07/07/2023,09/07/2023,27,2023,0.49,0,0,1,0,PENDIENTE,HERBICIDA,DRAGOCSON,DRAGOCSON 4X5 LT
842,H048,275.0,0.0,LIQUIDOS,IZUCAR,09/06/2023,11/06/2023,23,2023,0.00,0,0,1,0,PENDIENTE,HERBICIDA,DRAGOCSON,DRAGOCSON 4X5 LT
912,H048,4000.0,0.0,LIQUIDOS,IZUCAR,12/05/2023,14/05/2023,19,2023,0.00,0,0,1,0,PENDIENTE,HERBICIDA,DRAGOCSON,DRAGOCSON 4X5 LT
997,H048,1048.0,921.0,LIQUIDOS,IZUCAR,14/07/2023,16/07/2023,28,2023,0.88,0,0,1,0,PENDIENTE,HERBICIDA,DRAGOCSON,DRAGOCSON 4X5 LT
1060,H048,10200.0,6119.0,LIQUIDOS,IZUCAR,16/06/2023,18/06/2023,24,2023,0.60,0,0,1,0,PENDIENTE,HERBICIDA,DRAGOCSON,DRAGOCSON 4X5 LT
